In [14]:
import os
import io
import pickle
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

# Scopes necesarios para acceder a Drive
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

# Autenticación con token reutilizable
def authenticate_drive():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)
    return service

# Descargar solo imágenes nuevas
def download_images_from_folder(service, folder_id, output_dir='imagenes_descargadas'):
    os.makedirs(output_dir, exist_ok=True)

    # lista de archivos ya descargados
    archivos_locales = set(os.listdir(output_dir))

    # Buscar imágenes en la carpeta de Drive
    query = f"'{folder_id}' in parents and mimeType contains 'image/'"
    results = service.files().list(q=query, fields="files(id, name)").execute()
    items = results.get('files', [])

    if not items:
        print("No se encontraron imágenes en la carpeta.")
        return

    for item in items:
        file_id = item['id']
        file_name = item['name']

        if file_name in archivos_locales:
            print(f"Ya existe: {file_name}")
            continue

        print(f"Descargando: {file_name}")
        request = service.files().get_media(fileId=file_id)
        local_path = os.path.join(output_dir, file_name)
        fh = io.FileIO(local_path, 'wb')
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
        print(f"✅ Imagen guardada: {file_name}")

# Ejecutar el script
if __name__ == '__main__':
    folder_id = '1X8W66foKIso8NTygY5d0GRbps7gGfB8D'  # Reemplazá con el ID real de tu carpeta
    service = authenticate_drive()
    download_images_from_folder(service, folder_id)


Ya existe: dji_export_1650199192675.jpg
Ya existe: 20250221_105941.jpg
Descargando: LC-941.jpg
✅ Imagen guardada: LC-941.jpg
